# Embedding Demo

In [6]:
%pip install openai

%pip install numpy
import openai
import os
import numpy as np
openai.api_key  = os.environ['OPENAI_API_KEY']

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [41]:
def get_embedding(text: str, model="text-embedding-ada-002") -> np.ndarray:
    return openai.Embedding.create(input=[text], model=model)["data"][0]["embedding"]


In [3]:
r =get_embedding("I like to walk my dog")

In [30]:
%pip install torch

Note: you may need to restart the kernel to use updated packages.


In [35]:

model ='text-embedding-ada-002'
comp_x=['I love chocolate cake', 'bullish', 'love','i love apple', 'rise','solar eclipse' ]
comp_y =['Advanced quantum physics', 'bearish', 'hate','i hate apple','fall', 'Spaghetti carbonara']
print(get_embedding('bull', model=model))
for (x,y) in zip(comp_x, comp_y):
    emb1 = get_embedding(x)
    emb2 = get_embedding(y)
    # get norm of the vectors

    dot_product = np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
            
    print (f'{x} vs {y}  \nSimilarity: {dot_product}')

[-0.03699738532304764, -0.00023745006183162332, -0.015277997590601444, -0.012700089253485203, 0.002219301648437977, 0.00977710634469986, -0.032748233526945114, -0.006546263583004475, -0.013870636001229286, -0.04316812381148338, 0.010663473978638649, 0.027240576222538948, -0.0019063666695728898, -0.019432421773672104, -0.012057303451001644, 0.007754023652523756, 0.03109728917479515, 0.00939143542200327, 0.004431837238371372, -0.008586261421442032, -0.012124965898692608, 0.01453371997922659, 0.0009388052276335657, -0.008843376301229, -0.012821880169212818, -0.005663278978317976, 0.01902645267546177, -0.0023630827199667692, -0.014263072982430458, -0.013403770513832569, 0.006184273865073919, -0.0011341782519593835, -0.005930542480200529, -0.005578701850026846, -0.01975719816982746, -0.011563373729586601, 0.005521189887076616, -0.006262084469199181, 0.01004775334149599, 0.018241576850414276, 0.02576555125415325, 0.008890739642083645, 0.008078799583017826, -0.011644568294286728, -0.000658009

In [43]:
# do emebedding of king - man + woman -> queen
model ='text-embedding-ada-002'
def analogy(x1, x2, y1, model=model):
    emb1 = get_embedding(x1, model=model)
    emb2 = get_embedding(x2, model=model)
    emb3 = get_embedding(y1, model=model)
    print(type(emb1))
    # add the vectors which are np arrays
    emb4 = emb1 - emb2 + emb3
    return emb4

# get word from embedding
def get_word(emb, model=model):
    q = openai.Engine("davinci").search(embedding=emb, query="", max_rerank=100)["data"][0]["text"]
    return q

emb_q = analogy('king', 'man','woman',model=model)
print(get_word(emb_q, model=model))

<class 'list'>


NameError: name 'emb4' is not defined

In [27]:
%pip install transformers
% pip install torch

  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/ae/95/283a1c004430bd2a9425d6937fc545dd49a4e4592feb76be0299a14e2378/transformers-4.32.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 2.3 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for regex!=2019.12.17 from https://files.pythonhosted.org/packages/db/88/7c35fafe46fda07246c143a106f71880c3368ac5ef3beef89cc17b3bf12c/regex-2023.8.8-cp311-cp311-macosx_10_9_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 26.7 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for safetensors>=0.3.1 from https://files.pythonhosted.org/packages/6c/2f/6c52fc3bb1d5c3e0916e47fa1dc3920dff3748490c51789c3232a1513e86/safetensors-0.3.3-cp311-cp311-macosx_12_0_universal2.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 31.9 MB/s eta

UsageError: Line magic function `%` not found.


In [28]:
from transformers import GPT2Tokenizer, GPT2Model
import torch

def gpt2_embedding(sentence):
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    model = GPT2Model.from_pretrained('gpt2')
    
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    
    return outputs.last_hidden_state.mean(dim=1)  # Averaging embeddings for simplicity

def compare_embeddings_gpt2(list1, list2):
    dot_products = []
    for phrase1 in list1:
        for phrase2 in list2:
            emb1 = gpt2_embedding(phrase1)
            emb2 = gpt2_embedding(phrase2)
            dot_product = torch.dot(emb1, emb2) / (torch.norm(emb1) * torch.norm(emb2))
            dot_products.append((phrase1, phrase2, dot_product.item()))
    return dot_products

list1 = ["I love chocolate cake", "Advanced quantum physics"]
list2 = ["Solar eclipse", "Spaghetti carbonara"]

result = compare_embeddings_gpt2(list1, list2)
print(result)



RuntimeError: Failed to import transformers.models.gpt2.modeling_gpt2 because of the following error (look up to see its traceback):
dlopen(/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/lib/libtorch_global_deps.dylib, 0x000A): tried: '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/lib/libtorch_global_deps.dylib' (mach-o file, but is an incompatible architecture (have (arm64), need (x86_64)))